In [5]:
import random
import torch
from torch.utils.data import DataLoader, Dataset
import pandas as pd

from typing import  List
import os
from dotenv import load_dotenv
import databasemanager
import model_loading
from databasepipeline import DatabasePipeline

load_dotenv()


True

In [3]:
# For Reproducibility
random.seed(2024)
torch.manual_seed(2024)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [9]:
multi_gpu = os.getenv("multi_gpu")
a_name = os.getenv("a_name")
q_name = os.getenv("q_name")
t_name = os.getenv("t_name")
a_path = os.getenv("a_path")
q_path = os.getenv("q_path")
database_embedding = os.getenv("DATABASE_NAME")
collection_embedding = os.getenv("COLLECTION_NAME")

database_answer = os.getenv("DATABASE_NAME_1")
collection_answer = os.getenv("COLLECTION_NAME_1")

In [4]:
# Load Passage, Models and Tokenizers:
answers = pd.read_csv('qa/answers.csv')


# load_a = model_loading.load_model(a_path, a_name , device )
# load_q = model_loading.load_model(q_path, q_name, device )

load_a = model_loading.load_model_from_gpu(a_path, a_name, device)
load_q = model_loading.load_model_from_gpu(q_path, q_name, device)

a_enc = model_loading.enableMultiGPU(load_a, multi_gpu)
q_enc = model_loading.enableMultiGPU(load_q, multi_gpu)

tokenizer = model_loading.load_tokenizer(t_name)

model/a_encoder_model.bin


In [7]:
class AnswerDataset(Dataset):
    def __init__(self, answer: List[str]):
        self.answer = answer
    
    def __len__(self) -> int:
        return len(self.answer)
    
    def __getitem__(self, index) -> str:
        return self.answer[index]

batch_size = 64
answers['Answer'] = answers['Answer'].fillna('').str.replace('[^a-zA-Z0-9.!,]', ' ', regex=True).replace('\s+', ' ', regex=True)
answer = answers['Answer'].tolist()

answerDataset = AnswerDataset(answer)
answer_loader = DataLoader(answerDataset, batch_size = batch_size, shuffle = False)

In [14]:
database_pipeline = DatabasePipeline(q_enc, a_enc, tokenizer, answer_loader, device)

In [21]:
database_manager_embedding = databasemanager.DatabaseManager(database_embedding, collection_embedding)
database_manager_answer = databasemanager.DatabaseManager(database_answer, collection_answer)

In [22]:
database_pipeline.connection_db(database_manager_embedding, database_manager_answer)

In [ ]:
# Inject Data Now to an empty Database
database_pipeline.embed_passage()
database_pipeline.insertanswersdb()


In [20]:
# Once done, close the connection to the database:
database_pipeline.disconnect_db()